In [1]:
import codecs
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
from sklearn_crfsuite import scorers,metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate,train_test_split, cross_val_predict
import sklearn_crfsuite
import pandas as pd
import numpy as np
import csv
import pickle
import glob
import nltk
import re

In [2]:
#จัดการประโยคซ้ำ
data_not=[]
def Unique(p):
 text=re.sub("<[^>]*>","",p)
 text=re.sub("\[(.*?)\]","",text)
 text=re.sub("\[\/(.*?)\]","",text)
 if text not in data_not:
  data_not.append(text)
  return True
 else:
  return False
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')

In [3]:
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
 text=text.strip()
 text=re.sub("<[^>]*>","",text)
 text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
 text=re.sub("(\[\w+\])","***\\1",text)
 text2=[]
 for i in text.split('***'):
  if "[" in i:
   text2.append(i)
  else:
   text2.append("[word]"+i+"[/word]")
 text="".join(text2)#re.sub("[word][/word]","","".join(text2))
 return text.replace("[word][/word]","")
# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text) # นำไปใส่ tag [word]
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text) # แยก tag ออกมาจากข้อความ
    j=0
    conll2002="" # ประกาศตัวแปรเก็บ conll2002
    for tagopen,text,tagclose in tag: # ลูปใน tag โดยเป็น (tagopen,text,tagclose)
        word_cut=word_tokenize(text,engine="newmm") # ใช้ตัวตัดคำ newmm ของ PyThaiNLP
        i=0
        txt5=""
        while i<len(word_cut): #ลูปตามจำนวน token ที่ตัดในtag
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word': # ไม่เป็น tag [word] และเป็น i หรือตัวเริ่มต้น tag
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else: # เป็น [word]
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002) # เพิ่ม postag ใส่
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine="perceptron")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
	"""
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
	with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
		lines = f.read().splitlines()
	return [a for a in lines if Unique(a)] # เอาไม่ซ้ำกัน

def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n') # นำไปแปลงเป็น conll2002
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all

def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all
def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

In [4]:
def doc2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Features from current word
    features={
        'word.word': word,
        'word.isspace':word.isspace(),
        'postag':postag,
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        prevword = doc[i-1][0]
        postag1 = doc[i-1][1]
        features['word.prevword'] = prevword
        features['word.previsspace']=prevword.isspace()
        features['word.prepostag'] = postag1
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        postag1 = doc[i+1][1]
        features['word.nextword'] = nextword
        features['word.nextisspace']=nextword.isspace()
        features['word.nextpostag'] = postag1
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,postag,tag) in doc]

# Loading Model

In [5]:
model_name = 'model/300_NER_CRF_model.sav'
crf = pickle.load(open(model_name, 'rb'))

# Predict

In [6]:
data2=getall(get_data("unseen_noTag.txt"))
datatofile2=alldata_list(data2)

In [7]:
datatofile2

[[('สวัสดี', 'NCMN', 'O'),
  ('เพื่อน', 'NCMN', 'O'),
  ('ๆ', 'PUNC', 'O'),
  ('ครับ', 'NCMN', 'O'),
  (' ', 'PUNC', 'O'),
  ('ทริป', 'NCMN', 'O'),
  ('นี้', 'DDAC', 'O'),
  ('ไม่', 'NEG', 'O'),
  ('ได้', 'XVAM', 'O'),
  ('เตรียมตัว', 'VACT', 'O'),
  ('นาน', 'ADVN', 'O'),
  ('เลย', 'ADVN', 'O'),
  (' ', 'PUNC', 'O'),
  ('มี', 'VSTA', 'O'),
  ('เวลา', 'NCMN', 'O'),
  ('หา', 'VACT', 'O'),
  ('ข้อมูล', 'NCMN', 'O'),
  (' ', 'PUNC', 'O'),
  ('จอง', 'NCMN', 'O'),
  ('ตัว', 'CNIT', 'O'),
  (' ', 'PUNC', 'O'),
  ('หา', 'VACT', 'O'),
  ('ที่พัก', 'NCMN', 'O'),
  (' ', 'PUNC', 'O'),
  ('ไม่', 'NEG', 'O'),
  ('ถึง', 'RPRE', 'O'),
  ('อาทิตย์', 'NCMN', 'O'),
  (' ', 'PUNC', 'O'),
  ('ร่วม', 'VACT', 'O'),
  ('กลุ่ม', 'NCMN', 'O'),
  ('กัน', 'ADVN', 'O'),
  ('ได้', 'XVAE', 'O'),
  (' ', 'PUNC', 'O'),
  ('4', 'DCNM', 'O'),
  (' ', 'PUNC', 'O'),
  ('คน', 'CNIT', 'O'),
  (' ', 'PUNC', 'O'),
  ('ของ', 'RPRE', 'O'),
  ('วัน', 'NCMN', 'O'),
  ('จันทร์', 'NPRP', 'O'),
  (' ', 'PUNC', 'O'),
  ('ที่', 'PREL

In [8]:
len(datatofile2)

10

In [9]:
X_data2 = [extract_features(doc) for doc in datatofile2] # เอา คำ แยกออกมา

In [11]:
y_pred2 = crf.predict(X_data2)

In [12]:
len(y_pred2)

10

# Insert index to predicted data

In [13]:
data_label = []
for sentence_no in range(len(datatofile2)):
    index = 0
    for item in datatofile2[sentence_no]:
        item = list(item)
        item.insert(0, sentence_no+1)
        item.insert(1, index)
        item[4] = y_pred2[sentence_no][index]
#         item.insert(4, y_pred2[sentence_no][index])
        item = tuple(item)
        data_label.append(item)
        index += 1

print(len(data_label))

12708


In [14]:
data_label

[(1, 0, 'สวัสดี', 'NCMN', 'O'),
 (1, 1, 'เพื่อน', 'NCMN', 'O'),
 (1, 2, 'ๆ', 'PUNC', 'O'),
 (1, 3, 'ครับ', 'NCMN', 'O'),
 (1, 4, ' ', 'PUNC', 'O'),
 (1, 5, 'ทริป', 'NCMN', 'O'),
 (1, 6, 'นี้', 'DDAC', 'O'),
 (1, 7, 'ไม่', 'NEG', 'O'),
 (1, 8, 'ได้', 'XVAM', 'O'),
 (1, 9, 'เตรียมตัว', 'VACT', 'O'),
 (1, 10, 'นาน', 'ADVN', 'O'),
 (1, 11, 'เลย', 'ADVN', 'O'),
 (1, 12, ' ', 'PUNC', 'O'),
 (1, 13, 'มี', 'VSTA', 'O'),
 (1, 14, 'เวลา', 'NCMN', 'O'),
 (1, 15, 'หา', 'VACT', 'O'),
 (1, 16, 'ข้อมูล', 'NCMN', 'O'),
 (1, 17, ' ', 'PUNC', 'O'),
 (1, 18, 'จอง', 'NCMN', 'O'),
 (1, 19, 'ตัว', 'CNIT', 'O'),
 (1, 20, ' ', 'PUNC', 'O'),
 (1, 21, 'หา', 'VACT', 'O'),
 (1, 22, 'ที่พัก', 'NCMN', 'O'),
 (1, 23, ' ', 'PUNC', 'O'),
 (1, 24, 'ไม่', 'NEG', 'O'),
 (1, 25, 'ถึง', 'RPRE', 'O'),
 (1, 26, 'อาทิตย์', 'NCMN', 'O'),
 (1, 27, ' ', 'PUNC', 'O'),
 (1, 28, 'ร่วม', 'VACT', 'O'),
 (1, 29, 'กลุ่ม', 'NCMN', 'O'),
 (1, 30, 'กัน', 'ADVN', 'O'),
 (1, 31, 'ได้', 'XVAE', 'O'),
 (1, 32, ' ', 'PUNC', 'O'),
 (1, 33, '4',

# Merge label

In [15]:
i=0
loc = []
while(i < len(data_label)):
    if(data_label[i][4] == "B-LOCATION"):
        item = []
        item.append(data_label[i][0])
        item.append(data_label[i][1])
        word = data_label[i][2]
        while(data_label[i+1][4] == "I-LOCATION"):
            i += 1
            word += data_label[i][2]
        item.append(word)
        item.append("LOCATION")
        item = tuple(item)
    loc.append(item)
    i += 1
print(len(loc))

12420


In [17]:
loc_list = list(dict.fromkeys(loc))
loc_list = loc_list[1:]
loc_list

[(1, 48, 'เชียงใหม่', 'LOCATION'),
 (1, 281, 'ดอยอินทนนท์', 'LOCATION'),
 (1, 331, 'เชียงใหม่', 'LOCATION'),
 (1, 396, 'ถ.ห้วยแก้ว', 'LOCATION'),
 (1, 401, 'กาดสวนแก้ว', 'LOCATION'),
 (1, 406, 'ดอยอินทนนท์', 'LOCATION'),
 (1, 615, 'อ.จอมทอง', 'LOCATION'),
 (1, 641, 'อ.จอมทอง', 'LOCATION'),
 (1, 848, 'บ้านปกเปียง', 'LOCATION'),
 (1, 886, 'บ้านปกเปียง', 'LOCATION'),
 (1, 1174, 'น้ำตกวชิรธาน', 'LOCATION'),
 (1, 1185, 'เมืองเชียงใหม่', 'LOCATION'),
 (1, 1189, 'สันป่าตอง', 'LOCATION'),
 (1, 1311, 'เชียงใหม่', 'LOCATION'),
 (2, 2, 'ดอยแม่สลอง', 'LOCATION'),
 (2, 28, 'ดอยอ่างขาง', 'LOCATION'),
 (2,
  35,
  'อำเภอแม่อาย อำเภอวชิรปราการ อำเภอฝาง เขตจังหวัดเชียงใหม่',
  'LOCATION'),
 (2, 95, 'สวนส้มธนาธร', 'LOCATION'),
 (2, 158, 'ดอยอ่างขาง', 'LOCATION'),
 (2, 295, 'ดอยอ่างขาง', 'LOCATION'),
 (2, 365, 'ดอยอ่างขาง', 'LOCATION'),
 (2, 403, 'ภูชี้ฟ้า', 'LOCATION'),
 (2, 488, 'ชายแดนไทย', 'LOCATION'),
 (2, 493, 'พม่า', 'LOCATION'),
 (2, 503, 'ปะหร่อง(เชื้อสายพม่า', 'LOCATION'),
 (2, 532, 'พม่า', 'LO

In [18]:
len(loc_list)

194

In [19]:
loc_df = pd.DataFrame(loc_list, columns=["Thread", "Index", "Word", "Label"])
loc_df

,Thread,Index,Word,Label
0,1,48,เชียงใหม่,LOCATION
1,1,281,ดอยอินทนนท์,LOCATION
2,1,331,เชียงใหม่,LOCATION
3,1,396,ถ.ห้วยแก้ว,LOCATION
4,1,401,กาดสวนแก้ว,LOCATION
...,...,...,...,...
189,10,2270,เกาะผึ้ง,LOCATION
190,10,2309,เกาะผึ้ง,LOCATION
191,10,2314,เกาะหินงาม,LOCATION
192,10,2599,หลีเป๊ะ,LOCATION


In [20]:
# loc_df.to_csv('unseen_predict.csv', index=False)